# This notebook will use sequential model to predict the results in EPl matches

In [1]:
# Initial imports
import pandas as pd
import numpy as np
from pathlib import Path
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


2022-12-07 19:23:01.931768: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Reading in dataframe
df_epl = pd.read_csv(Path('Resources/epl_match_data.csv'))
# Displaying dataframe
df_epl

,Unnamed: 0,FTHG,FTAG,HTHG,HTAG,HTR,HS,AS,HST,AST,...,Referee_l Mason,Referee_ A D'Urso,Referee_ A Wiley,Referee_ C Foy,Referee_ D Gallagher,Referee_ H Webb,Referee_ M Atkinson,Referee_ N Barry,Referee_ S Dunn,Referee_ U Rennie
0,3215,2.30,0.75,0.95,0.30,1.45,16.85,6.60,9.70,3.15,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3220,2.40,0.75,1.00,0.30,1.45,17.00,6.35,9.65,2.95,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3253,2.20,0.65,0.95,0.20,1.50,17.25,6.35,9.80,2.95,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3284,2.25,0.75,0.90,0.25,1.40,17.25,6.50,9.70,3.10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3313,2.30,0.95,0.90,0.35,1.30,18.20,6.30,10.05,3.10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4700,7287,1.30,2.30,0.65,1.10,0.90,13.05,15.00,7.10,9.60,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4701,7309,1.25,2.30,0.55,1.00,0.90,13.10,15.20,7.30,9.65,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4702,7332,1.25,2.30,0.55,0.95,0.90,13.00,15.25,7.25,9.70,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4703,7348,1.20,2.35,0.50,1.00,0.80,12.75,15.35,7.30,9.90,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# Dropping unnecessary columns
df_epl = df_epl.drop(columns = ['Unnamed: 0'])
# Counting value types
df_epl.dtypes.value_counts()

float64    263
int64        2
dtype: int64

In [4]:
# Splitting dataframe into X and y dataframes
y = df_epl[['FTR']]
X= df_epl.drop(columns = ['FTR'])
# Displaying dataframes
display(y)
display(X)

,FTR
0,0
1,2
2,1
3,1
4,0
...,...
4700,0
4701,0
4702,0
4703,0


,FTHG,FTAG,HTHG,HTAG,HTR,HS,AS,HST,AST,HC,...,Referee_l Mason,Referee_ A D'Urso,Referee_ A Wiley,Referee_ C Foy,Referee_ D Gallagher,Referee_ H Webb,Referee_ M Atkinson,Referee_ N Barry,Referee_ S Dunn,Referee_ U Rennie
0,2.30,0.75,0.95,0.30,1.45,16.85,6.60,9.70,3.15,9.30,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.40,0.75,1.00,0.30,1.45,17.00,6.35,9.65,2.95,9.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.20,0.65,0.95,0.20,1.50,17.25,6.35,9.80,2.95,9.70,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2.25,0.75,0.90,0.25,1.40,17.25,6.50,9.70,3.10,9.55,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.30,0.95,0.90,0.35,1.30,18.20,6.30,10.05,3.10,9.60,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4700,1.30,2.30,0.65,1.10,0.90,13.05,15.00,7.10,9.60,6.60,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4701,1.25,2.30,0.55,1.00,0.90,13.10,15.20,7.30,9.65,6.90,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4702,1.25,2.30,0.55,0.95,0.90,13.00,15.25,7.25,9.70,6.55,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4703,1.20,2.35,0.50,1.00,0.80,12.75,15.35,7.30,9.90,6.75,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
display(X_train.shape)

(3528, 264)

In [23]:
# Create the StandardScaler instance
X_scaler = MinMaxScaler()

# Fit the scaler to the features training dataset
X_scaler.fit(X_train)

# Scale both the training and testing data from the features dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [24]:
# Define the the number of inputs to the model
number_inputs = 264

# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 = 256

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = 188


In [26]:
# Create the Sequential model instance
nn = Sequential()

# Add the first Dense layer specifying the number of inputs, the number of hidden nodes, and the activation function
nn.add(Dense(units=hidden_nodes_layer1, input_dim=number_inputs, activation="relu"))

# Add the second Dense layer specifying the number of hidden nodes and the activation function
nn.add(Dense(units=hidden_nodes_layer2, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(1))


In [27]:
# View size of the model
nn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 256)               67840     
                                                                 
 dense_13 (Dense)            (None, 188)               48316     
                                                                 
 dense_14 (Dense)            (None, 1)                 189       
                                                                 
Total params: 116,345
Trainable params: 116,345
Non-trainable params: 0
_________________________________________________________________


In [28]:
# Compile the Sequential model
nn.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])

In [29]:
# Fit the model
nn.fit(X_train_scaled,y_train, 
                    epochs=50,
                    batch_size=100,
                    shuffle=True)

Epoch 1/50
36/36 [==============================] - 3s 12ms/step - loss: 0.8497 - accuracy: 0.2548
Epoch 2/50
36/36 [==============================] - 1s 14ms/step - loss: 0.6218 - accuracy: 0.2548
Epoch 3/50
36/36 [==============================] - 0s 11ms/step - loss: 0.5986 - accuracy: 0.2548
Epoch 4/50
36/36 [==============================] - 1s 23ms/step - loss: 0.5730 - accuracy: 0.2560
Epoch 5/50
36/36 [==============================] - 0s 12ms/step - loss: 0.5433 - accuracy: 0.2574
Epoch 6/50
36/36 [==============================] - 1s 14ms/step - loss: 0.5051 - accuracy: 0.2656
Epoch 7/50
36/36 [==============================] - 0s 13ms/step - loss: 0.4587 - accuracy: 0.2817
Epoch 8/50
36/36 [==============================] - 0s 11ms/step - loss: 0.4088 - accuracy: 0.3016
Epoch 9/50
36/36 [==============================] - 0s 11ms/step - loss: 0.3475 - accuracy: 0.3271
Epoch 10/50
36/36 [==============================] - 0s 12ms/step - loss: 0.2930 - accuracy: 0.3532
Epoch 11/